In [4]:
# ================================================================
# 멀티센터 + 회귀/분류 동시 파이프라인
# ================================================================
import os, time, math, pickle, random
import numpy as np
import pandas as pd
from datetime import datetime

import torch
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    r2_score, mean_squared_error, mean_absolute_error,
    f1_score, accuracy_score, confusion_matrix
)
from sklearn.model_selection import ParameterGrid

from pytorch_tabnet.tab_model import TabNetRegressor, TabNetClassifier

# 맨 위쪽에 추가
import random
SEED = 42
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)


# ----------------------------
# 0) 환경/출력 디렉토리 설정
# ----------------------------
OUT_DIR = "../results_tabnet_corrected_v2"
os.makedirs(OUT_DIR, exist_ok=True)

# 기존 코드에 device 변수가 없다면 안전하게 감지
if "device" not in globals():
    device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
print(f"[Device] {device}")

# MPS면 pin_memory=False, 그 외엔 True
PIN_MEM = False if device == 'mps' else True


# GPU 세팅에 따른 배치/에폭 기본값
if device == 'mps':
    BATCH_REG = 256; VBS_REG = 64; EPOCH_REG = 400
    BATCH_CLS = 256; VBS_CLS = 64; EPOCH_CLS = 300
elif device == 'cuda':
    BATCH_REG = 512; VBS_REG = 128; EPOCH_REG = 600
    BATCH_CLS = 512; VBS_CLS = 128; EPOCH_CLS = 400
else:
    BATCH_REG = 256; VBS_REG = 64; EPOCH_REG = 250
    BATCH_CLS = 256; VBS_CLS = 64; EPOCH_CLS = 200

# ----------------------------
# 1) 공통 유틸
# ----------------------------
DROP_COLS_BASE = ['날짜','요일','1처리장','2처리장','정화조','중계펌프장','시설현대화','3처리장','4처리장']
EXCLUDE_REG = ['합계_1일후','합계_2일후','등급','등급_1일후','등급_2일후','합계']
EXCLUDE_CLS = ['합계','합계_2일후','등급','등급_2일후']  # 분류에선 '등급_1일후'가 타깃

def load_and_split(center_name, target_col, for_task="regression"):
    """센터 데이터를 로드하고 시계열 분할 및 스케일링을 수행.
    - for_task: 'regression' or 'classification'
    반환: dict(X_train_s, X_val_s, X_test_s, y_train, y_val, y_test, scalers, feature_names)
    """
    # 1) 로드
    path = f"../data/add_feature/{center_name}_add_feature.csv"
    df = pd.read_csv(path, encoding='utf-8-sig')
    # 2) 불필요 컬럼 제거
    df = df.drop([c for c in DROP_COLS_BASE if c in df.columns], axis=1)
    # 3) 결측 제거
    df = df.dropna().reset_index(drop=True)

    # 4) 피처/타깃 분리
    if target_col not in df.columns:
        raise KeyError(f"[{center_name}] Target '{target_col}' not found. available={list(df.columns)[:10]}...")

    if for_task == "regression":
        exclude = [c for c in EXCLUDE_REG if c in df.columns]
    else:
        exclude = [c for c in EXCLUDE_CLS if c in df.columns]

    X = df.drop(exclude + [target_col], axis=1, errors='ignore').copy()
    y = df[target_col].copy()

    # 분류 타깃을 int로 정리(0~3 가정)
    if for_task == "classification":
        y = y.astype(int)

    # 5) 시계열 70/20/10
    n = len(X)
    tr_end = int(n*0.7); va_end = int(n*0.9)
    X_train, X_val, X_test = X.iloc[:tr_end], X.iloc[tr_end:va_end], X.iloc[va_end:]
    y_train, y_val, y_test = y.iloc[:tr_end], y.iloc[tr_end:va_end], y.iloc[va_end:]

    # 6) 스케일링(누수방지): 회귀는 X, y 둘 다 / 분류는 X만
    x_scaler = StandardScaler()
    X_train_s = x_scaler.fit_transform(X_train).astype(np.float32)
    X_val_s   = x_scaler.transform(X_val).astype(np.float32)
    X_test_s  = x_scaler.transform(X_test).astype(np.float32)

    if for_task == "regression":
        y_scaler = StandardScaler()
        y_train_s = y_scaler.fit_transform(y_train.values.reshape(-1,1)).astype(np.float32)
        y_val_s   = y_scaler.transform(y_val.values.reshape(-1,1)).astype(np.float32)
        y_test_s  = y_scaler.transform(y_test.values.reshape(-1,1)).astype(np.float32)
        scalers = dict(x=x_scaler, y=y_scaler)
        y_pack  = (y_train, y_val, y_test, y_train_s, y_val_s, y_test_s)
    else:
        scalers = dict(x=x_scaler, y=None)
        y_pack  = (y_train, y_val, y_test, None, None, None)

    return dict(
        X_train_s=X_train_s, X_val_s=X_val_s, X_test_s=X_test_s,
        y_pack=y_pack, feature_names=X.columns.tolist(), n_samples=n, splits=(len(X_train), len(X_val), len(X_test)),
        scalers=scalers
    )

def regression_metrics(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return dict(
        R2=r2_score(y_true, y_pred),
        RMSE=rmse,
        MAE=mean_absolute_error(y_true, y_pred),
        MAPE=(np.mean(np.abs((y_true - y_pred)/(y_true + 1e-8)))*100),
        SMAPE=(np.mean(2*np.abs(y_true - y_pred)/(np.abs(y_true)+np.abs(y_pred)+1e-8))*100),
    )

def classification_metrics(y_true, y_pred):
    return dict(
        Accuracy=accuracy_score(y_true, y_pred),
        F1_weighted=f1_score(y_true, y_pred, average='weighted'),
        F1_macro=f1_score(y_true, y_pred, average='macro'),
        CM=confusion_matrix(y_true, y_pred)
    )

def compute_class_weight(y_train):
    """클래스 불균형 자동 가중치 계산(단순 역빈도)."""
    vals, cnts = np.unique(y_train, return_counts=True)
    freq = cnts / cnts.sum()
    weight = {int(k): float(1.0/(f+1e-8)) for k, f in zip(vals, freq)}
    # 평균이 1 되도록 정규화
    m = np.mean(list(weight.values()))
    weight = {k: v/m for k,v in weight.items()}
    return weight

# ----------------------------
# 2) 하이퍼파라미터 그리드
# ----------------------------
grid_reg = {
    'n_d': [32, 64],
    'n_a': [32, 64],
    'n_steps': [4, 5],
    'gamma': [1.0, 1.2],
    'lambda_sparse': [1e-4, 1e-5],
    'learning_rate': [0.001, 0.002, 0.005],
    'weight_decay': [1e-5, 1e-6],
}
grid_cls = {
    'n_d': [24, 32, 48],
    'n_a': [24, 32, 48],
    'n_steps': [3, 4, 5],
    'gamma': [1.0, 1.3],
    'lambda_sparse': [1e-4, 1e-5],
    'learning_rate': [0.001, 0.002, 0.003],
    'weight_decay': [1e-5, 1e-6],
}

# 샘플링(시간 제한)
def sample_grid(grid, max_trials=20, seed=42):
    combos = list(ParameterGrid(grid))
    random.seed(seed)
    if len(combos) > max_trials:
        combos = random.sample(combos, max_trials)
    return combos

# ----------------------------
# 3) 트레이닝 루틴
# ----------------------------
def tune_tabnet_regression(pack, max_epochs=EPOCH_REG, batch=BATCH_REG, vbatch=VBS_REG, patience=30):
    Xtr, Xva, Xte = pack['X_train_s'], pack['X_val_s'], pack['X_test_s']
    # y_*_s 가 (n, 1) 2D인 상태로 들어오게 됨
    y_train, y_val, y_test, ytr_s, yva_s, yte_s = pack['y_pack']
    y_scaler = pack['scalers']['y']

    trials = sample_grid(grid_reg, max_trials=20)
    best_score = -np.inf; best_params=None; best_model=None
    t0 = time.time()

    for i, p in enumerate(trials, 1):
        print(f"[REG] trial {i}/{len(trials)} params={p}")
        model = TabNetRegressor(
            n_d=p['n_d'], n_a=p['n_a'], n_steps=p['n_steps'],
            gamma=p['gamma'], lambda_sparse=p['lambda_sparse'],
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=p['learning_rate'], weight_decay=p['weight_decay']),
            device_name=device, seed=42, verbose=0
        )
        # ✅ ytr_s, yva_s 모두 (n,1) 2D
        model.fit(
            X_train=Xtr, y_train=ytr_s,
            eval_set=[(Xva, yva_s)],
            eval_metric=['rmse'],
            max_epochs=max_epochs, patience=patience,
            batch_size=batch, virtual_batch_size=vbatch,
            num_workers=0, drop_last=False,
            pin_memory=PIN_MEM,  # ✅ 추가
        )

        # 예측도 (n,1) → inverse_transform 후 1D로 평탄화
        yva_pred_s = model.predict(Xva)                        # (n,1)
        yva_pred   = y_scaler.inverse_transform(yva_pred_s)    # (n,1)
        yva_pred   = yva_pred.ravel()                          # (n,)

        val_r2 = r2_score(y_val.values, yva_pred)
        if val_r2 > best_score:
            best_score, best_params, best_model = val_r2, p, model
            print("  -> new best on val R²:", round(val_r2, 4))

    train_time = time.time() - t0

    # 테스트 성능
    yte_pred_s = best_model.predict(Xte)                       # (n,1)
    yte_pred   = y_scaler.inverse_transform(yte_pred_s).ravel()
    reg_metrics = regression_metrics(y_test.values, yte_pred)

    return best_model, best_params, best_score, train_time, dict(y_true=y_test.values, y_pred=yte_pred), reg_metrics


def tune_tabnet_classification(pack, n_classes=4, max_epochs=EPOCH_CLS, batch=BATCH_CLS, vbatch=VBS_CLS, patience=30):
    Xtr, Xva, Xte = pack['X_train_s'], pack['X_val_s'], pack['X_test_s']
    y_train, y_val, y_test, *_ = pack['y_pack']

    trials = sample_grid(grid_cls, max_trials=20)
    class_weight = compute_class_weight(y_train.values)
    # TabNetClassifier는 class_weights 리스트/np.array 형식 기대
    weights_list = np.array([class_weight.get(i,1.0) for i in range(n_classes)], dtype=np.float32)

    best_score = -np.inf; best_params=None; best_model=None
    t0 = time.time()

    for i, p in enumerate(trials, 1):
        print(f"[CLS] trial {i}/{len(trials)} params={p}")
        model = TabNetClassifier(
            n_d=p['n_d'], n_a=p['n_a'], n_steps=p['n_steps'],
            gamma=p['gamma'], lambda_sparse=p['lambda_sparse'],
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=p['learning_rate'], weight_decay=p['weight_decay']),
            device_name=device, seed=42, verbose=0,
            # 불균형 클래스 가중치
            class_weights=weights_list
        )
        model.fit(
            X_train=Xtr, y_train=y_train.values,
            eval_set=[(Xva, y_val.values)],
            eval_metric=['accuracy'],
            max_epochs=max_epochs, patience=patience,
            batch_size=batch, virtual_batch_size=vbatch,
            num_workers=0, drop_last=False,
            pin_memory=PIN_MEM,  # ✅ 추가
        )
        # 검증 성능(F1_weighted)
        yva_pred = model.predict(Xva).ravel()
        f1w = f1_score(y_val.values, yva_pred, average='weighted')
        if f1w > best_score:
            best_score = f1w; best_params = p; best_model = model
            print("  -> new best on val F1w:", round(f1w,4))

    train_time = time.time()-t0
    # 테스트 성능
    yte_pred = best_model.predict(Xte).ravel()
    cls_metrics = classification_metrics(y_test.values, yte_pred)

    return best_model, best_params, best_score, train_time, dict(y_true=y_test.values, y_pred=yte_pred), cls_metrics

# ----------------------------
# 4) 실행 오케스트레이션
# ----------------------------
centers = ['nanji','jungnang','seonam','tancheon']  # 필요 시 추가
summary_rows = []  # 센터/과제별 요약 집계

for center in centers:
    print("\n" + "="*70)
    print(f"[{center.upper()}] 파이프라인 시작")
    print("="*70)

    # ===== 회귀: 합계_1일후 =====
    pack_reg = load_and_split(center, target_col='합계_1일후', for_task='regression')
    reg_model, reg_params, reg_val_best, reg_time, reg_pred, reg_metrics = tune_tabnet_regression(pack_reg)

    # 결과/모델 저장
    center_dir = os.path.join(OUT_DIR, center)
    os.makedirs(center_dir, exist_ok=True)

    reg_pickle = {
        'task': 'regression',
        'model': reg_model,
        'params': reg_params,
        'metrics': reg_metrics,
        'feature_names': pack_reg['feature_names'],
        'scalers': pack_reg['scalers'],
        'predictions': reg_pred
    }
    with open(os.path.join(center_dir, f"{center}_tabnet_reg.pkl"), "wb") as f:
        pickle.dump(reg_pickle, f)

    pd.DataFrame([{
        'center': center, 'task': 'regression', 'val_best_R2': reg_val_best,
        'R2': reg_metrics['R2'], 'RMSE': reg_metrics['RMSE'], 'MAE': reg_metrics['MAE'],
        'MAPE': reg_metrics['MAPE'], 'SMAPE': reg_metrics['SMAPE'],
        'training_time_sec': reg_time, 'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }]).to_csv(os.path.join(center_dir, f"{center}_tabnet_reg_results.csv"),
               index=False, encoding='utf-8-sig')

    summary_rows.append({
        'center': center, 'task': 'regression',
        **reg_metrics, 'training_time_sec': reg_time
    })

    print(f"[{center}] REG done: R2={reg_metrics['R2']:.4f}, RMSE={reg_metrics['RMSE']:.3f}")

    # ===== 분류: 등급_1일후 (0~3 다중분류) =====
    pack_cls = load_and_split(center, target_col='등급_1일후', for_task='classification')
    cls_model, cls_params, cls_val_best, cls_time, cls_pred, cls_metrics = tune_tabnet_classification(pack_cls, n_classes=4)

    # 결과/모델 저장
    cls_pickle = {
        'task': 'classification',
        'model': cls_model,
        'params': cls_params,
        'metrics': {k:v for k,v in cls_metrics.items() if k!='CM'},
        'confusion_matrix': cls_metrics['CM'],
        'feature_names': pack_cls['feature_names'],
        'scalers': pack_cls['scalers'],
        'predictions': cls_pred
    }
    with open(os.path.join(center_dir, f"{center}_tabnet_cls.pkl"), "wb") as f:
        pickle.dump(cls_pickle, f)

    # 혼동행렬은 따로 CSV로 저장
    pd.DataFrame(cls_metrics['CM']).to_csv(os.path.join(center_dir, f"{center}_tabnet_cls_confusion_matrix.csv"),
                                           index=False, encoding='utf-8-sig')

    pd.DataFrame([{
        'center': center, 'task': 'classification', 'val_best_F1w': cls_val_best,
        'Accuracy': cls_metrics['Accuracy'], 'F1_weighted': cls_metrics['F1_weighted'],
        'F1_macro': cls_metrics['F1_macro'],
        'training_time_sec': cls_time, 'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }]).to_csv(os.path.join(center_dir, f"{center}_tabnet_cls_results.csv"),
               index=False, encoding='utf-8-sig')

    summary_rows.append({
        'center': center, 'task': 'classification',
        'Accuracy': cls_metrics['Accuracy'],
        'F1_weighted': cls_metrics['F1_weighted'],
        'F1_macro': cls_metrics['F1_macro'],
        'training_time_sec': cls_time
    })

    print(f"[{center}] CLS done: Acc={cls_metrics['Accuracy']:.3f}, F1w={cls_metrics['F1_weighted']:.3f}")

# ----------------------------
# 5) 전체 요약 저장
# ----------------------------
summary_df = pd.DataFrame(summary_rows)
summary_path = os.path.join(OUT_DIR, "multi_center_summary.csv")
summary_df.to_csv(summary_path, index=False, encoding='utf-8-sig')
print("\n✅ 전체 요약 저장:", summary_path)

# 사람이 보기 좋게 출력
print("\n[요약] 회귀 성능(R²) by center")
print(summary_df[summary_df['task']=='regression'][['center','R2','RMSE','MAE','MAPE']])

print("\n[요약] 분류 성능(F1_weighted) by center")
print(summary_df[summary_df['task']=='classification'][['center','Accuracy','F1_weighted','F1_macro']])


[Device] mps

[NANJI] 파이프라인 시작
[REG] trial 1/20 params={'gamma': 1.2, 'lambda_sparse': 1e-05, 'learning_rate': 0.002, 'n_a': 32, 'n_d': 32, 'n_steps': 5, 'weight_decay': 1e-06}

Early stopping occurred at epoch 80 with best_epoch = 50 and best_val_0_rmse = 0.88896


/opt/anaconda3/envs/youngwon/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


  -> new best on val R²: 0.5082
[REG] trial 2/20 params={'gamma': 1.0, 'lambda_sparse': 0.0001, 'learning_rate': 0.002, 'n_a': 64, 'n_d': 64, 'n_steps': 4, 'weight_decay': 1e-05}

Early stopping occurred at epoch 182 with best_epoch = 152 and best_val_0_rmse = 0.83399


/opt/anaconda3/envs/youngwon/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


  -> new best on val R²: 0.5672
[REG] trial 3/20 params={'gamma': 1.0, 'lambda_sparse': 0.0001, 'learning_rate': 0.001, 'n_a': 32, 'n_d': 64, 'n_steps': 5, 'weight_decay': 1e-05}

Early stopping occurred at epoch 96 with best_epoch = 66 and best_val_0_rmse = 0.94417


...

/opt/anaconda3/envs/youngwon/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


[REG] trial 20/20 params={'gamma': 1.0, 'lambda_sparse': 1e-05, 'learning_rate': 0.001, 'n_a': 64, 'n_d': 32, 'n_steps': 5, 'weight_decay': 1e-06}

Early stopping occurred at epoch 194 with best_epoch = 164 and best_val_0_rmse = 0.89412


/opt/anaconda3/envs/youngwon/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


[nanji] REG done: R2=0.3707, RMSE=74913.024
[CLS] trial 1/20 params={'gamma': 1.0, 'lambda_sparse': 0.0001, 'learning_rate': 0.003, 'n_a': 24, 'n_d': 32, 'n_steps': 3, 'weight_decay': 1e-05}


TypeError: __init__() got an unexpected keyword argument 'class_weights'

In [5]:
# Cell 1: 환경/라이브러리 & 시드/디바이스
import os, time, math, pickle, random
import numpy as np
import pandas as pd
from datetime import datetime

import torch
import torch.nn.functional as F

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    r2_score, mean_squared_error, mean_absolute_error,
    f1_score, accuracy_score, confusion_matrix, precision_recall_fscore_support
)
from sklearn.model_selection import ParameterGrid

from pytorch_tabnet.tab_model import TabNetRegressor, TabNetClassifier
import matplotlib.pyplot as plt

# 재현성
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# 디바이스
device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
PIN_MEM = False if device == 'mps' else True
print(f"[Device] {device}, pin_memory={PIN_MEM}")

if device == 'cuda':
    try: torch.set_float32_matmul_precision('high')
    except: pass


[Device] mps, pin_memory=False


In [6]:
# Cell 2: 경로/하이퍼파라미터/글로벌 설정
OUT_DIR = "../results_tabnet_corrected_v2"
os.makedirs(OUT_DIR, exist_ok=True)

DROP_COLS_BASE = ['날짜','요일','1처리장','2처리장','정화조','중계펌프장','시설현대화','3처리장','4처리장']
EXCLUDE_REG = ['합계_1일후','합계_2일후','등급','등급_1일후','등급_2일후','합계']
EXCLUDE_CLS = ['합계','합계_2일후','등급','등급_2일후']

# 디바이스별 배치/에폭
if device == 'mps':
    BATCH_REG = 256; VBS_REG = 64; EPOCH_REG = 400
    BATCH_CLS = 256; VBS_CLS = 64; EPOCH_CLS = 300
elif device == 'cuda':
    BATCH_REG = 512; VBS_REG = 128; EPOCH_REG = 600
    BATCH_CLS = 512; VBS_CLS = 128; EPOCH_CLS = 400
else:
    BATCH_REG = 256; VBS_REG = 64; EPOCH_REG = 250
    BATCH_CLS = 256; VBS_CLS = 64; EPOCH_CLS = 200

# 하이퍼파라미터 그리드(샘플링 예정)
grid_reg = {
    'n_d': [32, 64],
    'n_a': [32, 64],
    'n_steps': [4, 5],
    'gamma': [1.0, 1.2],
    'lambda_sparse': [1e-4, 1e-5],
    'learning_rate': [0.001, 0.002, 0.005],
    'weight_decay': [1e-5, 1e-6],
}
grid_cls = {
    'n_d': [24, 32, 48],
    'n_a': [24, 32, 48],
    'n_steps': [3, 4, 5],
    'gamma': [1.0, 1.3],
    'lambda_sparse': [1e-4, 1e-5],
    'learning_rate': [0.001, 0.002, 0.003],
    'weight_decay': [1e-5, 1e-6],
}

def sample_grid(grid, max_trials=20, seed=SEED):
    combos = list(ParameterGrid(grid))
    random.seed(seed)
    if len(combos) > max_trials:
        combos = random.sample(combos, max_trials)
    return combos


In [7]:
# Cell 3: 유틸 함수
def load_and_split(center_name, target_col, for_task="regression"):
    path = f"../data/add_feature/{center_name}_add_feature.csv"
    df = pd.read_csv(path, encoding='utf-8-sig')
    df = df.drop([c for c in DROP_COLS_BASE if c in df.columns], axis=1)
    df = df.dropna().reset_index(drop=True)

    if target_col not in df.columns:
        raise KeyError(f"[{center_name}] Target '{target_col}' not found. available={list(df.columns)[:12]}...")

    exclude = EXCLUDE_REG if for_task == "regression" else EXCLUDE_CLS
    exclude = [c for c in exclude if c in df.columns]

    X = df.drop(exclude + [target_col], axis=1, errors='ignore').copy()
    y = df[target_col].copy()
    if for_task == "classification":
        y = y.astype(int)

    n = len(X); tr_end = int(n*0.7); va_end = int(n*0.9)
    X_train, X_val, X_test = X.iloc[:tr_end], X.iloc[tr_end:va_end], X.iloc[va_end:]
    y_train, y_val, y_test = y.iloc[:tr_end], y.iloc[tr_end:va_end], y.iloc[va_end:]

    x_scaler = StandardScaler()
    X_train_s = x_scaler.fit_transform(X_train).astype(np.float32)
    X_val_s   = x_scaler.transform(X_val).astype(np.float32)
    X_test_s  = x_scaler.transform(X_test).astype(np.float32)

    if for_task == "regression":
        y_scaler = StandardScaler()
        y_train_s = y_scaler.fit_transform(y_train.values.reshape(-1,1)).astype(np.float32)
        y_val_s   = y_scaler.transform(y_val.values.reshape(-1,1)).astype(np.float32)
        y_test_s  = y_scaler.transform(y_test.values.reshape(-1,1)).astype(np.float32)
        scalers = dict(x=x_scaler, y=y_scaler)
        y_pack  = (y_train, y_val, y_test, y_train_s, y_val_s, y_test_s)
    else:
        scalers = dict(x=x_scaler, y=None)
        y_pack  = (y_train, y_val, y_test, None, None, None)

    return dict(
        X_train_s=X_train_s, X_val_s=X_val_s, X_test_s=X_test_s,
        y_pack=y_pack,
        feature_names=X.columns.tolist(),
        n_samples=n, splits=(len(X_train), len(X_val), len(X_test)),
        scalers=scalers
    )

def regression_metrics(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return dict(
        R2=r2_score(y_true, y_pred),
        RMSE=rmse,
        MAE=mean_absolute_error(y_true, y_pred),
        MAPE=np.mean(np.abs((y_true - y_pred)/(y_true + 1e-8)))*100,
        SMAPE=np.mean(2*np.abs(y_true - y_pred)/(np.abs(y_true)+np.abs(y_pred)+1e-8))*100,
    )

def classification_metrics(y_true, y_pred):
    return dict(
        Accuracy=accuracy_score(y_true, y_pred),
        F1_weighted=f1_score(y_true, y_pred, average='weighted'),
        F1_macro=f1_score(y_true, y_pred, average='macro'),
        CM=confusion_matrix(y_true, y_pred)
    )

def compute_class_weight(y_train):
    vals, cnts = np.unique(y_train, return_counts=True)
    freq = cnts / cnts.sum()
    weight = {int(k): float(1.0/(f+1e-8)) for k, f in zip(vals, freq)}
    m = np.mean(list(weight.values()))
    weight = {k: v/m for k,v in weight.items()}
    return weight

def make_weighted_ce(weight_arr):
    w_cpu = torch.tensor(weight_arr, dtype=torch.float32)
    def _loss_fn(inputs, targets):
        return F.cross_entropy(inputs, targets, weight=w_cpu.to(inputs.device))
    return _loss_fn

# (선택) 학습 히스토리 파서: dict/History 객체 모두 대응
def to_history_dict(h):
    if isinstance(h, dict): return h
    if hasattr(h, "history") and isinstance(h.history, dict): return h.history
    try: return dict(h)
    except: pass
    try:
        v = vars(h)
        if isinstance(v, dict):
            if "history" in v and isinstance(v["history"], dict):
                return v["history"]
            return v
    except: pass
    return {}


In [9]:
# Cell 4: 트레이닝 루틴
def tune_tabnet_regression(pack, max_epochs=EPOCH_REG, batch=BATCH_REG, vbatch=VBS_REG, patience=30):
    Xtr, Xva, Xte = pack['X_train_s'], pack['X_val_s'], pack['X_test_s']
    y_train, y_val, y_test, ytr_s, yva_s, yte_s = pack['y_pack']
    y_scaler = pack['scalers']['y']

    trials = sample_grid(grid_reg, max_trials=20)
    best_score = -np.inf; best_params=None; best_model=None
    t0 = time.time()

    for i, p in enumerate(trials, 1):
        print(f"[REG] trial {i}/{len(trials)} params={p}")
        model = TabNetRegressor(
            n_d=p['n_d'], n_a=p['n_a'], n_steps=p['n_steps'],
            gamma=p['gamma'], lambda_sparse=p['lambda_sparse'],
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=p['learning_rate'], weight_decay=p['weight_decay']),
            device_name=device, seed=SEED, verbose=0,
            mask_type='entmax'
        )
        model.fit(
            X_train=Xtr, y_train=ytr_s,
            eval_set=[(Xva, yva_s)],
            eval_metric=['rmse'],
            max_epochs=max_epochs, patience=patience,
            batch_size=batch, virtual_batch_size=vbatch,
            num_workers=0, drop_last=False,
            pin_memory=PIN_MEM,
        )
        yva_pred_s = model.predict(Xva)
        yva_pred   = y_scaler.inverse_transform(yva_pred_s).ravel()
        val_r2 = r2_score(y_val.values, yva_pred)
        if val_r2 > best_score:
            best_score = val_r2; best_params = p; best_model = model
            print("  -> new best on val R²:", round(val_r2, 4))

    train_time = time.time() - t0
    yte_pred_s = best_model.predict(Xte)
    yte_pred   = y_scaler.inverse_transform(yte_pred_s).ravel()
    reg_metrics = regression_metrics(y_test.values, yte_pred)

    return best_model, best_params, best_score, train_time, dict(y_true=y_test.values, y_pred=yte_pred), reg_metrics


def tune_tabnet_classification(pack, max_epochs=EPOCH_CLS, batch=BATCH_CLS, vbatch=VBS_CLS, patience=30, base_n_classes=4):
    Xtr, Xva, Xte = pack['X_train_s'], pack['X_val_s'], pack['X_test_s']
    y_train, y_val, y_test, *_ = pack['y_pack']

    classes = np.unique(y_train.values.astype(int))
    n_classes = max(int(classes.max()) + 1, base_n_classes)

    class_weight = compute_class_weight(y_train.values)
    weights_list = np.array([class_weight.get(i, 1.0) for i in range(n_classes)], dtype=np.float32)
    loss_fn = make_weighted_ce(weights_list)

    trials = sample_grid(grid_cls, max_trials=20)
    best_score = -np.inf; best_params=None; best_model=None
    t0 = time.time()

    for i, p in enumerate(trials, 1):
        print(f"[CLS] trial {i}/{len(trials)} params={p}")
        model = TabNetClassifier(
            n_d=p['n_d'], n_a=p['n_a'], n_steps=p['n_steps'],
            gamma=p['gamma'], lambda_sparse=p['lambda_sparse'],
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=p['learning_rate'], weight_decay=p['weight_decay']),
            device_name=device, seed=SEED, verbose=0,
        )
        model.fit(
            X_train=Xtr, y_train=y_train.values.astype(int),
            eval_set=[(Xva, y_val.values.astype(int))],
            eval_metric=['accuracy'],
            loss_fn=loss_fn,
            max_epochs=max_epochs, patience=patience,
            batch_size=batch, virtual_batch_size=vbatch,
            num_workers=0, drop_last=False,
            pin_memory=PIN_MEM,
        )
        yva_pred = model.predict(Xva).ravel().astype(int)
        f1w = f1_score(y_val.values.astype(int), yva_pred, average='weighted')
        if f1w > best_score:
            best_score = f1w; best_params = p; best_model = model
            print("  -> new best on val F1w:", round(f1w, 4))

    train_time = time.time()-t0
    yte_pred = best_model.predict(Xte).ravel().astype(int)
    cls_metrics = classification_metrics(y_test.values.astype(int), yte_pred)

    return best_model, best_params, best_score, train_time, dict(y_true=y_test.values, y_pred=yte_pred), cls_metrics


In [10]:
# Cell 5: 오케스트레이션 실행
centers = ['nanji','jungnang','seonam','tancheon']
summary_rows = []

for center in centers:
    print("\n" + "="*75)
    print(f"[{center.upper()}] 파이프라인 시작")
    print("="*75)

    try:
        # 회귀
        pack_reg = load_and_split(center, target_col='합계_1일후', for_task='regression')
        reg_model, reg_params, reg_val_best, reg_time, reg_pred, reg_metrics = tune_tabnet_regression(
            pack_reg, max_epochs=EPOCH_REG, batch=BATCH_REG, vbatch=VBS_REG, patience=30
        )
        center_dir = os.path.join(OUT_DIR, center); os.makedirs(center_dir, exist_ok=True)
        with open(os.path.join(center_dir, f"{center}_tabnet_reg.pkl"), "wb") as f:
            pickle.dump({
                'task': 'regression', 'model': reg_model, 'params': reg_params,
                'metrics': reg_metrics, 'feature_names': pack_reg['feature_names'],
                'scalers': pack_reg['scalers'], 'predictions': reg_pred
            }, f)
        pd.DataFrame([{
            'center': center, 'task': 'regression', 'val_best_R2': reg_val_best,
            'R2': reg_metrics['R2'], 'RMSE': reg_metrics['RMSE'], 'MAE': reg_metrics['MAE'],
            'MAPE': reg_metrics['MAPE'], 'SMAPE': reg_metrics['SMAPE'],
            'training_time_sec': reg_time, 'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }]).to_csv(os.path.join(center_dir, f"{center}_tabnet_reg_results.csv"), index=False, encoding='utf-8-sig')
        summary_rows.append({'center': center, 'task':'regression', **reg_metrics, 'training_time_sec': reg_time})
        print(f"[{center}] REG done: R2={reg_metrics['R2']:.4f}, RMSE={reg_metrics['RMSE']:.3f}")

        # 분류
        pack_cls = load_and_split(center, target_col='등급_1일후', for_task='classification')
        cls_model, cls_params, cls_val_best, cls_time, cls_pred, cls_metrics = tune_tabnet_classification(
            pack_cls, max_epochs=EPOCH_CLS, batch=BATCH_CLS, vbatch=VBS_CLS, patience=30, base_n_classes=4
        )
        with open(os.path.join(center_dir, f"{center}_tabnet_cls.pkl"), "wb") as f:
            pickle.dump({
                'task': 'classification', 'model': cls_model, 'params': cls_params,
                'metrics': {k:v for k,v in cls_metrics.items() if k!='CM'},
                'confusion_matrix': cls_metrics['CM'],
                'feature_names': pack_cls['feature_names'],
                'scalers': pack_cls['scalers'], 'predictions': cls_pred
            }, f)
        cm_df = pd.DataFrame(cls_metrics['CM'],
                             index=[f"true_{i}" for i in range(cls_metrics['CM'].shape[0])],
                             columns=[f"pred_{i}" for i in range(cls_metrics['CM'].shape[1])])
        cm_df.to_csv(os.path.join(center_dir, f"{center}_tabnet_cls_confusion_matrix.csv"), index=True, encoding='utf-8-sig')
        pd.DataFrame([{
            'center': center, 'task': 'classification', 'val_best_F1w': cls_val_best,
            'Accuracy': cls_metrics['Accuracy'], 'F1_weighted': cls_metrics['F1_weighted'],
            'F1_macro': cls_metrics['F1_macro'],
            'training_time_sec': cls_time, 'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }]).to_csv(os.path.join(center_dir, f"{center}_tabnet_cls_results.csv"), index=False, encoding='utf-8-sig')
        summary_rows.append({'center': center, 'task':'classification',
                             'Accuracy': cls_metrics['Accuracy'],
                             'F1_weighted': cls_metrics['F1_weighted'],
                             'F1_macro': cls_metrics['F1_macro'],
                             'training_time_sec': cls_time})
        print(f"[{center}] CLS done: Acc={cls_metrics['Accuracy']:.3f}, F1w={cls_metrics['F1_weighted']:.3f}")

    except Exception as e:
        print(f"[{center}] ❌ 실패: {e}")
        continue

summary_df = pd.DataFrame(summary_rows)
summary_path = os.path.join(OUT_DIR, "multi_center_summary.csv")
summary_df.to_csv(summary_path, index=False, encoding='utf-8-sig')
print("\n✅ 전체 요약 저장:", summary_path)

# 확인 출력
print("\n[요약] 회귀 성능(R²) by center")
if (summary_df['task']=='regression').any():
    display(summary_df[summary_df['task']=='regression'][['center','R2','RMSE','MAE','MAPE']])

print("\n[요약] 분류 성능(F1_weighted) by center")
if (summary_df['task']=='classification').any():
    display(summary_df[summary_df['task']=='classification'][['center','Accuracy','F1_weighted','F1_macro']])



[NANJI] 파이프라인 시작
[REG] trial 1/20 params={'gamma': 1.2, 'lambda_sparse': 1e-05, 'learning_rate': 0.002, 'n_a': 32, 'n_d': 32, 'n_steps': 5, 'weight_decay': 1e-06}

Early stopping occurred at epoch 143 with best_epoch = 113 and best_val_0_rmse = 0.8691


/opt/anaconda3/envs/youngwon/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


  -> new best on val R²: 0.53
[REG] trial 2/20 params={'gamma': 1.0, 'lambda_sparse': 0.0001, 'learning_rate': 0.002, 'n_a': 64, 'n_d': 64, 'n_steps': 4, 'weight_decay': 1e-05}

Early stopping occurred at epoch 91 with best_epoch = 61 and best_val_0_rmse = 0.91679


/opt/anaconda3/envs/youngwon/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


[REG] trial 3/20 params={'gamma': 1.0, 'lambda_sparse': 0.0001, 'learning_rate': 0.001, 'n_a': 32, 'n_d': 64, 'n_steps': 5, 'weight_decay': 1e-05}

Early stopping occurred at epoch 166 with best_epoch = 136 and best_val_0_rmse = 0.87802


...

/opt/anaconda3/envs/youngwon/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


[CLS] trial 14/20 params={'gamma': 1.0, 'lambda_sparse': 0.0001, 'learning_rate': 0.002, 'n_a': 48, 'n_d': 24, 'n_steps': 5, 'weight_decay': 1e-06}

Early stopping occurred at epoch 137 with best_epoch = 107 and best_val_0_accuracy = 0.94788


/opt/anaconda3/envs/youngwon/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


[CLS] trial 15/20 params={'gamma': 1.0, 'lambda_sparse': 1e-05, 'learning_rate': 0.002, 'n_a': 24, 'n_d': 32, 'n_steps': 3, 'weight_decay': 1e-06}


KeyboardInterrupt: 

In [ ]:
# Cell 6: 회귀 결과 시각화 (센터 1개 선택)
center_to_plot = 'nanji'  # 변경 가능
center_dir = os.path.join(OUT_DIR, center_to_plot)

with open(os.path.join(center_dir, f"{center_to_plot}_tabnet_reg.pkl"), "rb") as f:
    reg_obj = pickle.load(f)

y_true = reg_obj['predictions']['y_true']
y_pred = reg_obj['predictions']['y_pred']
metrics = reg_obj['metrics']
feature_names = reg_obj['feature_names']
tabnet_reg = reg_obj['model']

print(f"[{center_to_plot.upper()}][REG] R2={metrics['R2']:.4f}, RMSE={metrics['RMSE']:.1f}, MAE={metrics['MAE']:.1f}")

# 1) 예측 vs 실제
plt.figure(figsize=(6,6))
plt.scatter(y_true, y_pred, alpha=0.6)
mn, mx = min(y_true.min(), y_pred.min()), max(y_true.max(), y_pred.max())
plt.plot([mn,mx],[mn,mx],'--')
plt.title(f"{center_to_plot.upper()} 예측 vs 실제 (R²={metrics['R2']:.3f})")
plt.xlabel("실제값"); plt.ylabel("예측값"); plt.grid(alpha=0.3)
plt.show()

# 2) 잔차
resid = y_true - y_pred
plt.figure(figsize=(6,4))
plt.scatter(y_pred, resid, alpha=0.6)
plt.axhline(0, ls='--', c='r')
plt.title("잔차 플롯"); plt.xlabel("예측값"); plt.ylabel("잔차"); plt.grid(alpha=0.3)
plt.show()

# 3) 최근 구간 시계열(뒤 50개)
n_show = min(50, len(y_true))
idx = np.arange(len(y_true))[-n_show:]
plt.figure(figsize=(10,4))
plt.plot(idx, y_true[-n_show:], label='실제')
plt.plot(idx, y_pred[-n_show:], ls='--', label='예측')
plt.fill_between(idx, y_true[-n_show:], y_pred[-n_show:], alpha=0.2)
plt.title(f"시계열 결과 (최근 {n_show}개)")
plt.xlabel("시점"); plt.ylabel("값"); plt.legend(); plt.grid(alpha=0.3)
plt.show()

# 4) Feature Importance (Top 15)
if hasattr(tabnet_reg, 'feature_importances_'):
    fi = tabnet_reg.feature_importances_
    fi_df = pd.DataFrame({'Feature': feature_names, 'Importance': fi}).sort_values('Importance', ascending=False).head(15)
    plt.figure(figsize=(8,6))
    plt.barh(range(len(fi_df)), fi_df['Importance'])
    plt.yticks(range(len(fi_df)), fi_df['Feature'])
    plt.gca().invert_yaxis()
    plt.title("Feature Importance (Top 15)")
    plt.xlabel("Importance")
    plt.tight_layout()
    plt.show()


In [ ]:
# Cell 7: 학습 곡선 (회귀 모델 히스토리)
hist = getattr(tabnet_reg, 'history', None)
if hist is None:
    print("history 없음")
else:
    H = to_history_dict(hist)
    print("history keys:", list(H.keys()))

    # 키 후보
    loss_key_candidates = ["loss","train_loss","training_loss","loss_epoch","train_loss_epoch"]
    val_key_candidates  = ["val_loss","val_0_rmse","valid_0_rmse","validation_0_rmse","valid_rmse","valid_loss"]

    def pick_first(d, cands):
        for k in cands:
            if k in d: return k
        return None

    lk, vk = pick_first(H, loss_key_candidates), pick_first(H, val_key_candidates)
    if lk is None and vk is None:
        print("usable loss/val keys not found")
    else:
        plt.figure(figsize=(7,4))
        if lk is not None: plt.plot(H[lk], label='train')
        if vk is not None: plt.plot(H[vk], label='valid')
        plt.title("학습 곡선"); plt.xlabel("epoch"); plt.ylabel("loss/metric"); plt.legend(); plt.grid(alpha=0.3)
        plt.show()

        plt.figure(figsize=(7,4))
        if lk is not None: plt.semilogy(H[lk], label='train')
        if vk is not None: plt.semilogy(H[vk], label='valid')
        plt.title("학습 곡선 (log)"); plt.xlabel("epoch"); plt.ylabel("loss/metric (log)"); plt.legend(); plt.grid(alpha=0.3)
        plt.show()

        if hasattr(tabnet_reg, 'best_epoch'):
            print("best_epoch:", tabnet_reg.best_epoch)


In [ ]:
# Cell 8: 분류 결과 시각화 (센터 1개 선택)
with open(os.path.join(center_dir, f"{center_to_plot}_tabnet_cls.pkl"), "rb") as f:
    cls_obj = pickle.load(f)

y_true_cls = cls_obj['predictions']['y_true'].astype(int)
y_pred_cls = cls_obj['predictions']['y_pred'].astype(int)
m_cls = cls_obj['metrics']
cm = cls_obj['confusion_matrix']

print(f"[{center_to_plot.upper()}][CLS] Acc={m_cls['Accuracy']:.3f}, F1w={m_cls['F1_weighted']:.3f}, F1m={m_cls['F1_macro']:.3f}")

# 혼동행렬
plt.figure(figsize=(5,4))
plt.imshow(cm, interpolation='nearest')
plt.title("Confusion Matrix"); plt.xlabel("Pred"); plt.ylabel("True")
plt.colorbar(); plt.xticks(range(cm.shape[1])); plt.yticks(range(cm.shape[0]))
plt.tight_layout(); plt.show()

# 클래스별 PRF(옵션)
prec, rec, f1, support = precision_recall_fscore_support(y_true_cls, y_pred_cls, labels=np.unique(y_true_cls), zero_division=0)
cls_table = pd.DataFrame({
    'class': np.unique(y_true_cls),
    'precision': prec, 'recall': rec, 'f1': f1, 'support': support
})
display(cls_table)


In [ ]:
# Cell 9: 전체 요약 시각화
summary_df = pd.read_csv(os.path.join(OUT_DIR, "multi_center_summary.csv"), encoding='utf-8-sig')

# 회귀 R²
reg_df = summary_df[summary_df['task']=='regression'].copy()
if not reg_df.empty:
    plt.figure(figsize=(7,4))
    plt.bar(reg_df['center'], reg_df['R2'])
    plt.title("센터별 회귀 R²"); plt.xlabel("center"); plt.ylabel("R²"); plt.ylim(0, 1)
    for x,y in zip(reg_df['center'], reg_df['R2']):
        plt.text(x, y+0.01, f"{y:.2f}", ha='center')
    plt.show()

# 분류 F1-weighted
cls_df = summary_df[summary_df['task']=='classification'].copy()
if not cls_df.empty:
    plt.figure(figsize=(7,4))
    plt.bar(cls_df['center'], cls_df['F1_weighted'])
    plt.title("센터별 분류 F1-weighted"); plt.xlabel("center"); plt.ylabel("F1-weighted"); plt.ylim(0, 1)
    for x,y in zip(cls_df['center'], cls_df['F1_weighted']):
        plt.text(x, y+0.01, f"{y:.2f}", ha='center')
    plt.show()
